# DSC 540 Final Project

[1. Abstract](#abstract)

[2. Introduction](#introduction)
[3. Exploratory Analysis Part 1 (Preprocessing and Data Cleaning)](#packages)

* [3 a. Packages](#packages)
* [3 b. Explaining the Data ](#explaining_data)
* [3 c. Record Counts ](#counts)
* [3 d. Missing Values ](#missing_value)
* [3 e. Data Dictionary ](#data_dictionary)
* [3 f. Dropped and Created Features ](#dropped_created)
* [3 g. Statistical Summary ](#statistical_summary)
* [3 h. KNN Imputation for Monthly Rent Payment Variable ](#knn_imputation)
* [3 i. Distribution of Target Variable](#pie_chart)
* [3 j. Correlation Matrix](#correlation_matrix)
* [3 k. Education](#education)
* [3 l. Exploring Regional Difference](#regions)
* [3 m. Urban vs. Rural](#urban_vs_rural)
* [3 n. Gender Differences](#gender)
* [3 o. Overcrowding](#overcrowding)
* [3 p. House Quality (and some feature engineering)](#house_quality)
* [3 q. Dependency](#dependency)
* [3 r. Age](#age)
* [3 s. Distribution of Monthly Rent Payments](#rent)
* [3 t. Distribution of Missing Rent Values](#missing_rent)

[4. Data Reduction Techniques (PCA, KPCA, Feature Selection)](#pca)
* [4 a. PCA](#pca)
* [4 b. Kernel PCA](#kpca)
* [4 c. Feature Selection Using Feature Importance From a Logistic Regression Model](#feature_selection)

[5. Research Questions](#research_questions)
* [5 a. Research Question 1: Can we use PCA to identify the main components that determine poverty level?](#research_question_1)
* [5 b. Research Question 2: Can we develop a model better than our baseline model?](#research_question_2)
* [5 c. Research Question 3: How important are regional differences?](#research_question_3)

[6. Machine Learning](#models)
* [6 a. Train-Test Split](#models)
* [6 b. Logistic Regression](#logistic_regression)
* [6 c. AdaBoost](#adaboost)
* [6 d. Gaussian Naive Bayes](#gnb)
* [6 e. Decision Tree](#decision_tree)
* [6 f. Random Forest](#random_forest)
* [6 g. KNN](#knn)
* [6 h. Neural Network](#neural)
* [6 i. Stochastic Gradient Descent](#sgd)
* [6 j. Support Vector Classifier](#svc)
* [6 k. Rebalancing Techniques](#rebalance)





[7 a. Experiment Results](#experiment_results)

[7 b. Experiment Results for Rebalancing Techniques](#experiment_results_b)

[8. Conclusion](#conclusion)

[9. Contributions](#contributions)
* [9 a. Max Rodrigues](#max_rodrigues)
* [9 b. Andrew Schiek](#andrew_schiek)
* [9 c. Xintong Li](#Xintong_Li)
* [9 c. Mashall Jahangir](#Mashall_Jahangir)

[10. References](#references)





<a id='abstract'></a>

# 1. Abstract
Social programs have a difficulty determining which households are in most desperate need of welfare assistance. Many of the poorest households do not have the records (salary slips, tax returns, etc) to indicate that they are deserving of welfare assistance. Currently, machine learning models that utilize data that describe observable characteristics of homes help to ameliorate this issue. These models are called Proxy Means Tests (PMTs). Performance using these limited features is often inconsistent, especially as populations grow.

The data used for this project is from a Kaggle competition, and the purpose of the competition was to determine if the use of non-traditional econometric features (such as the level of education the head of the household has) can help in determine the households that most need welfare assistance. After preprocessing the data (feature engineering, scaling, PCA, etc), we applied the full data to multiple models and also applied rebalancing techniques to some of these models. We also took a subset of the data to mimic a typical dataset used for PMTs and compared the results. We found that the most important features are actually observable ones, but the inclusion of non-traditional features improves model performance. Data were highly imbalanced, so balancing techniques typically helped with performance. KNN also had strong performance due to its resilience to handle imbalanced data. Our findings indicate that the improvement in metrics with the inclusion of non-traditional features may not be worth the many long man-hours involved in collecting this difficult to obtain data if these techniques are to be applied in other countries.


<a id='introduction'></a>

# 2. Introduction
Many social programs around the world face an issue of assessing social welfare needs (many of the poorest people often do not have the records to indicate they qualify). A workaround to this issue is through the use of machine learning: rather than using income verification records or tax returns, models use a Proxy Means Test (PMT) to assess income qualification. These models use observable features of households (e.g wall or ceiling quality) as an approximation of income level. However, these models are often inaccurate and become unstable as populations change over time.

For this project, we are using data from a Kaggle competition (/www.kaggle.com/c/costa-rican-household-poverty-prediction/data?select=train.csv) to predict how impoverished a household is. Data collectors from the Inter-American Development Bank classified a large number of households (9,557) in Costa Rica into four groups: 

1 = extreme poverty

2 = moderate poverty

3 = vulnerable households

4 = non vulnerable household

In addition to features typically included in a PMT, data regarding information about individuals in the household (e.g. level of education, number of people living in the home, etc) was included with the hopes of improving predictive performance without the need of actual financial records. For this problem, we plan to use a variety of multiclass classification techniques to determine a model that has a high overall performance (F-1 macro average) and identifies extreme & moderate poverty households well (precision, recall and F-1 scores will be used as evaluative measures for these classes). Given the imbalanced nature of the dataset, we will also try rebalancing techniques (under/over-sampling).

In addition to finding a best model, we also want to determine the effectiveness of including non-traditional variables in a PMT model. We used both the entire dataset for prediction as well as a subset of the data that are the features typically included in PMT models. We are also interested in identifying the most important features for predicting poverty to see if non-traditional PMT features are better or worse predictors than than traditional PMT features. The conclusions developed from this analysis can help determine best approaches for assessing social welfare needs for other countries by finding a best model and determining the types of features that should be collected. We will also apply our best model to the Kaggle holdout test set to see how our model performs relative to other Kagglers.



<a id='packages'></a>
## 3. Exploratory Data Analysis Part 1 (Data Cleaning and Preprocessing)
### 3 a. Packages



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn import preprocessing
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from collections import OrderedDict
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import tree
import graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import KernelPCA
from sklearn.inspection import permutation_importance
from sklearn.impute import KNNImputer
from sklearn.feature_selection import RFE
from sklearn.svm import SVC



#from google.colab import files 
#uploaded = files.upload()

#import io 
#data = pd.read_csv(io.StringIO(uploaded["train.csv"].decode('utf-8')),sep=',')
data = pd.read_csv("../input/rs-costa-rican-household-poverty-level-prediction/train.csv")
data.head()

<a id='explaining_data[link text](https://)'></a>

## 3 b. Explain your data (Explanatory & Target Variable)


The data are a mixture of categorical, numeric and one-hot encoded variables with a Target variable coded by the Inter-American Developmental Bank indicating the level of poverty a household is facing (distribution of the target variable is below and summaries of the explanatory variables are included in the EDA).




In [ ]:
data.Target.value_counts()

<a id='counts[link text](https://)'></a>

## 3 c. Record Counts

In [ ]:
print("The data have", data.shape[0], "rows and",data.shape[1],"columns")

<a id='missing_values[link text](https://)'></a>

## 3 d. Missing Values
Below we identified that several columns have large amounts of missing values. There is likely a non-random nature to some of these missing values (espcially monthly rent payment since financial records are typically harder to obtain for more impoverished households). We created a binary variable ("rent_missing") indicating if the rent payment value was missing for an observation. We used KNN imputation to fill in the missing values. This allows us to retain the information of non-missing values of these rows and gain potential information of the non-random nature of the distribution of these missing values.



In [ ]:
total_missing_values = 0
for i in data.columns:
    if sum(data[i].isnull()) > 0:
        print("Missing values for", i,":",sum(data[i].isnull()))
    total_missing_values += sum(data[i].isnull())
print("Total missing values:", total_missing_values)

"v18q1" indicates the number of tablets the household has and "rez_esc" indicates the number of years behind in school the homeowner is. While these are both potentially useful variables, we have a number of education variables that are not missing available for use. As for the number of tablets, this feature more closely aligns with a categorical feature than a numeric and imputation techniques will likely be biased (given approximately 80% of values are missing for this feature). We therefore dropped "v18q1" and "rez_esc".


In [ ]:
data["rent_missing"] = np.where(data["v2a1"].isnull(),1,0)
#reverse one-hot encode to group by region to get monthly rent averages to fill missing values
regions_df = data[['lugar1','lugar2','lugar3','lugar4','lugar5','lugar6']]
warnings.filterwarnings("ignore")
regions_df = data[['lugar1','lugar2','lugar3','lugar4','lugar5','lugar6']]
regions_df['regions']='' # to create an empty column
regions_df2 = pd.get_dummies(regions_df).idxmax(1)

data["regions"] = regions_df2
data["meaneduc"] = data.groupby("regions").transform(lambda x: x.fillna(x.mean()))['meaneduc']
data["SQBmeaned"] = data.groupby("regions").transform(lambda x: x.fillna(x.mean()))['SQBmeaned']
data = data.drop(columns=['regions'])
data = data.drop(columns=['rez_esc'])
data = data.drop(columns=['v18q1'])
warnings.resetwarnings()


<a id='data_dictionary[link text](https://)'></a>

## 3 e. Data Dictionary

##Traditional PMT Features:

**Variables Related to House Size**: tamhog, hhsize

**Variables Describing Materials of Wall, Floor, or Roof** : paredblolad, paredzocalo, paredpreb, pareddes , paredmad, paredzinc, paredfibras, paredother, pisomoscer, pisocemento, pisoother, pisonatur, pisonotiene, pisomadera, techozinc, techoentrepiso, techocane, techootro", cielorazo

**Variables Describing Quality of Wall, Floor, or Roof (Good,Avg,Poor)**:
epared1, epared2, epared3, etecho1, etecho2, etecho3,
eviv1, eviv2, eviv3
- as a note, these were converted into variables indicating the quality of the house as a whole (house_poor_quality, house_average_quality, house_good_quality)


**Features Describing Region Household is Located**: lugar1, lugar2, lugar3, lugar4, lugar5, lugar6, urban


##Non-Traditional PMT Features (Features that typically require verification from households to obtain):

**Monthly rent payment**: v2a1

**Features Decribing Rent/Mortgage Payment**:  
tipovivi1 (fully paid), tipovivi2 (paying in installments), tipovivi3 (rented), tipovivi4 (precarious), tipovivi5 (borrowed)

**Features Indicating Overcrowding**: hacdor, hacapo,overcrowding

**Features Indicating Types of Technology Owned**: v18q (tablet),computer, television, mobilephone ,qmobilephone (# of phones)

**Features Describing Status of Head of Household**: status, estadocivil1, estadocivil2, estadocivil3, estadocivil4, estadocivil5, estadocivil6, estadocivil7, parentesco1, parentesco2,parentesco3, parentesco4, parentesco5, parentesco6, parentesco7, parentesco8, parentesco9, parentesco10, parentesco11, parentesco12

**Features Describing Number of Individuals Living in Household (Within Different Age Groups)**: hogar_nin (# age 0-19), hogar_adul(# adults), hogar_mayor (#adults 65+), hogar_total (total #), dependency (# under 19 + # 64+/# between 19 and 64)]

**Age of Head of Household**: age

**Features Describing Education Level of Head of Household & Members of the Household**: edjefe, edjefa, meaneduc, instlevel1, instlevel2, instlevel3, instlevel4, instlevel5, instlevel6, instlevel7, instlevel8, instlevel9, escolari

**Features Indicating Type of Water Provision**: abastaguadentro (inside), abastaguafuera (outside), abastaguano(none)

**Features Indicating Electricity Source**: electricity (public), planpri (private), noelec (none), coopele (from coopeartive)

**Features Describing Plumbing**: sanitario1 (no toilet), 
sanitario2 (toilet connected to sewer), sanitario3, (septic tank),sanitario5 (toiled connected to black hole),
sanitario6 (other), v14a (has a bathroom)

**Features Describing How Trash is Disposed**: elimbasu1 (truck), elimbasu2 (buried), elimbasu3 (burning), elimbasu4 (throwing in unoccupied space), elimbasu5 (other)

**Features Describing Number of Rooms the Household Has**: rooms (total #), bedrooms (# of bedrooms)

**Features Describing Number of People Living in the Household**: r4h1 (# males under 12), r4h2 (# males over 12), r4h3 (total # males), r4m1 (# females under 12), r4m2 (# females over 12), r4t1 (total # females), r4t1 (total # under 12), r4t2 ( total # over 12), r4t3 (total # in household), tamviv (total # in household)

**Does the house have a refrigerator?**: refrig

##Dropped Features
**Years behind in school**: rez_esc 

**Total Number of Tablets in the Household**: v18q1


<a id='dropped_created[link text](https://)'></a>

## 3 f. Dropped and Created Features

**Created (Feature Engineered)**
* Redundant variables converted to single variable (male/female -> gender), area1/area2 -> urban
* "yes/no" variables made numeric for analysis
* binary variable indicating if the rent variable is missing
* house quality features made instead of aspects of the house (e.g. if the house has a poor quality roof, wall or floor then the house is considered poor quality. Same for medium or good quality roof, wall or floor). The features used for feature engineering are dropped later prior to building models but are included for EDA.

**Dropped**
* ID variables
* elimbasu5 (rubbish disposed of by discarding in river) all values were 0
* dropped variables that are other variables squared


In [ ]:
data = data.drop(columns=['ID_num']) #dont need id for modeling
data = data.drop(columns=['idhogar']) #household Id
data = data.drop(columns=['elimbasu5']) #all values are zero
#male and female are redundant. Making one gender variable
data['gender'] = data['male']
data = data.drop(columns=['male'])
data = data.drop(columns=['female'])

#rural and urban variables are redundant. making one variable
data['urban'] = data['area1']
data = data.drop(columns=['area1'])
data = data.drop(columns=['area2'])

#dropping feature engineered variables... these are other features squared
data = data.drop(columns=['SQBescolari'])
data = data.drop(columns=['SQBage'])
data = data.drop(columns=['SQBhogar_total'])
data = data.drop(columns=['SQBedjefe'])
data = data.drop(columns=['SQBhogar_nin'])
data = data.drop(columns=['SQBovercrowding'])
data = data.drop(columns=['SQBdependency'])
data = data.drop(columns=['SQBmeaned'])
data = data.drop(columns=['agesq'])

#converting variables to be numeric
data['edjefe'] = data.edjefe.replace('yes', 1) 
data['edjefe'] = data.edjefe.replace('no', 0)

data['dependency'] = data.edjefe.replace('yes', 1)
data['dependency'] = data.edjefe.replace('no', 0)

data['edjefa'] = data.edjefe.replace('yes', 1)
data['edjefa'] = data.edjefe.replace('no', 0)

data["dependency"] = pd.to_numeric(data["dependency"])
data["edjefe"] = pd.to_numeric(data["edjefe"])
data["edjefa"] = pd.to_numeric(data["edjefa"])

house_poor_quality = data["epared1"] + data["etecho1"] + data["eviv1"]
data["house_poor_quality"] = np.where(house_poor_quality>0,1,0)

house_medium_quality = data["epared2"] + data["etecho2"] + data["eviv2"]
data["house_medium_quality"] = np.where(house_medium_quality>0,1,0)

house_good_quality = data["epared3"] + data["etecho3"] + data["eviv3"]
data["house_good_quality"] = np.where(house_good_quality>0,1,0)

X = data.drop('Target',axis=1)
y = data["Target"]

<a id='statistical_summary[link text](https://)'></a>

## 3 g. Statistical Summary

In [ ]:
pd.set_option('display.max_columns', 150)
X.describe()

We can inspect differences in mean between poor quality houses and average to good quality houses. We can see differences in most attributes. Most notably, rent payments ("v2a1") average 114,228 cólones whereas good/average home rent payments average 172,366 cólones.

In [ ]:
X.groupby("house_poor_quality").mean()

<a id='knn_imputation[link text](https://)'></a>

## 3 h. KNN Imputation for Monthly Rent Payment Variable

KNN imputation uses a KNN algorithm to fill missing values. We used n = 3.

In [ ]:
imputer = KNNImputer(n_neighbors=3)
df_filled = imputer.fit_transform(X)
X["v2a1"] = df_filled.T[0]

Now there are no missing values in the data.

In [ ]:
total_missing_values = 0
for i in X.columns:
    if sum(X[i].isnull()) > 0:
        print("Missing values for", i,":",sum(X[i].isnull()))
    total_missing_values += sum(X[i].isnull())
print("Total missing values:", total_missing_values)

<a id='pie_chart'></a>

## 3 i. Distribution of Target Variable


The pie chart shows that the majority of people are non vulnerable households i.e 62.7%.
Only 7.9% people fall into extreme poverty and 16.7% fall into moderate poverty when looked at their income distribution.
This indicates we need to consider imbalanced classes in model construction


In [ ]:
# Creating plots 
grouped = pd.DataFrame(data.groupby(['Target'])['r4t3'].count())
grouped.rename(columns={'r4t3':'Target_count'}, inplace=True)
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(122)
    
# Create colour palette
labels = ['extreme poverty', 'moderate poverty', 'vulnerable households', 'non vulnerable households']
theme = plt.get_cmap('Blues')
ax.set_prop_cycle("color", [theme(1. * i / len(labels))
                                 for i in range(len(labels))])
sns.set(font_scale=1.25)

# Create pie chart
pie = ax.pie(grouped['Target_count'],
                 autopct='%1.1f%%',
                 shadow=True,
                 startangle=20,
                 pctdistance=1.115,
                 explode=(0.1, 0.1, 0.1, 0.1))
   
# Turn pie chart into a donut chart
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Display donut plot with legend
plt.legend(pie[0], labels, loc="lower left")
ax.set_title('Income distribution among People surveyed \n', fontsize=15)
plt.tight_layout()
plt.show()

# <a id='correlation_matrix'></a>
## 3 j. Correlation Matrix

For the correlation matrix, I'm taking all variables that are not binary (or one-hot encoded) and putting these variables into a new dataframe to find all correlations.

In [ ]:
#for the correlation heatmap
non_binary_df = pd.DataFrame()
for i in X.columns:
    if max(X[i]) != 1:
        non_binary_df[i] = X[i]

In [ ]:
train_corr = non_binary_df.iloc[:,:-1]
correlations = train_corr.corr(method="pearson")
correlations
mask = np.zeros_like(correlations)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(30, 15))
    ax = sns.heatmap(correlations, mask=mask, vmax=1, square=True, cmap="YlGnBu", annot = True)

<a id='education'></a>

## 3 k. Education

The highest correlation with v2a1 (monthly rent payment) is meadeduc (average years of education for adults). Looking at the scatterplot, there seems to be a slight increase in rent as education goes up up to about 25 years of education

In [ ]:
correlation = X["v2a1"].corr(X["meaneduc"]) 
plt.scatter(X["meaneduc"], X["v2a1"], alpha=0.5)
plt.xlabel("Average Years of Education for Adults in the Household")
plt.ylabel("Monthly Rent Payment")
plt.title("Correlation of " + str(round(correlation,4)))
plt.show()

The plot below is the same as the previous plot but only goes to a meaneducation of 25. This more clearly demonstrates a positive correlation between monthly rent payments and education level.

In [ ]:
under_20_years_educ = X[X["meaneduc"] < 25]

correlation = under_20_years_educ["v2a1"].corr(under_20_years_educ["meaneduc"]) 
plt.scatter(under_20_years_educ["meaneduc"], under_20_years_educ["v2a1"], alpha=0.5)
plt.xlabel("Average Years of Education for Adults in the Household")
plt.ylabel("Monthly Rent Payment")
plt.title("Correlation of " + str(round(correlation,4)))
plt.show()

From the boxplots below we can see, Mean education of males and females in household is low in extreme poverty and high in non-vulnerable. The total population of male and female is also moderately high given their education in non-vulnerable income levels. 

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = sns.boxplot(x="Target", y="meaneduc", hue='r4h3', data=data, palette="Set3").set_title('Mean Education by Income levels and Total males in the household \n')

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = sns.boxplot(x="Target", y="meaneduc", hue='r4m3', data=data, palette="Set3").set_title('Mean Education by Income levels and Total females in the household \n')

<a id='regions'></a>

## 3 l. Regional Differences
We can look at monthly rent payment averages for the different regions. Below we see lugar1 is the most expensive while lugar6 is the least expensive. This implies lugar6 will have more impoverished people.

In [ ]:
for i in range(7):
    for j in X.columns:
        if j == "lugar" + str(i):
            print("Average monthly rent payment for region lugar" +str(i),":",X.groupby('lugar'+str(i), as_index=False)['v2a1'].mean().iloc[1,1])

In [ ]:
warnings.filterwarnings("ignore")

regions_df = data[['lugar1','lugar2','lugar3','lugar4','lugar5','lugar6']]
regions_df['regions']='' # to create an empty column
regions_df2 = pd.get_dummies(regions_df).idxmax(1)

X["regions"] = regions_df2
 
X["regions"] = regions_df["regions"]
warnings.resetwarnings()


In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.lmplot(x="meaneduc", y="v2a1",row = "regions",col = "urban", data=X)

<a id='urban_vs_rural'></a>

## 3 m. Urban vs Rural

The urban variable may be useful in identifying target=4 ("non-vulnerable households"). There is not a significant difference in the proportion of urban vs rural for the other targets (see below for actual proportions).

In [ ]:
ax = sns.countplot(x=y,hue=X["urban"])
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)

In [ ]:
cross_tab_urban_target = pd.crosstab(y, X["urban"], dropna=False)
props = cross_tab_urban_target[1]/(cross_tab_urban_target[0]+cross_tab_urban_target[1])
counter = 1
for i in props:
    print("Proportion of Urban dwellers in target",counter,":", i)
    counter+=1
    

<a id='gender'></a>

## 3 n. Gender Differences

For gender, 0 indicates female and 1 indicates male. Women seem to be more likely to be impoverished than men since the differences in bar sizes are greater for classes 1 and 2.

In [ ]:
ax = sns.countplot(x=y,hue=X["gender"])
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Gender Distribution for Each Income Level")

<a id='overcrowding'></a>

## 3 o. Overcrowding

It is difficult to distinguish differences in overcrowding with the countplot below. However, when we calculate the proportion of homes with overcrowding for each target, we can see that lower income households have higher levels of overcrowding (7.5% for the most impoverished class compared to <1% for the least impoverished class)

In [ ]:
ax = sns.countplot(x=y,hue=X["hacapo"])
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Overcrowding of Rooms for Each Income Level")

In [ ]:
cross_tab_overcrowding_target = pd.crosstab(y, X["hacapo"], dropna=False)
props = cross_tab_overcrowding_target[1]/(cross_tab_overcrowding_target[0]+cross_tab_overcrowding_target[1])
counter = 1
for i in props:
    print("Proportion of overcrowding in target",counter,":", i)
    counter+=1
    

<a id='house_quality'></a>

## 3 p. Distribution of Feature Engineered House Quality Variables

The bar charts below demonstrate that the quality of the house is a strong predictor of the level of poverty.

In [ ]:
ax = sns.countplot(x=y,hue=X.house_poor_quality)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Count of Homes with Either a Poor Quality Wall, Roof or Floor for Each Income Level")

In [ ]:
cross_tab_house_quality_target = pd.crosstab(y, X.house_poor_quality, dropna=False)
props = cross_tab_house_quality_target[1]/(cross_tab_house_quality_target[0]+cross_tab_house_quality_target[1])
counter = 1
for i in props:
    print("Proportion poor house quality in each target",counter,":", i)
    counter+=1

In [ ]:
ax = sns.countplot(x=y,hue=X.house_medium_quality)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Count of Homes with Either a Poor Quality Wall, Roof or Floor for Each Income Level")

In [ ]:
cross_tab_house_medium_quality_target = pd.crosstab(y, X.house_medium_quality, dropna=False)
props = cross_tab_house_medium_quality_target[1]/(cross_tab_house_medium_quality_target[0]+cross_tab_house_medium_quality_target[1])
counter = 1
for i in props:
    print("Proportion medium house quality in each target",counter,":", i)
    counter+=1

In [ ]:
ax = sns.countplot(x=y,hue=X.house_good_quality)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Count of Homes with Either a Good Quality Wall, Roof or Floor for Each Income Level")

In [ ]:
cross_tab_house_good_quality_target = pd.crosstab(y, X.house_good_quality, dropna=False)
props = cross_tab_house_good_quality_target[1]/(cross_tab_house_good_quality_target[0]+cross_tab_house_good_quality_target[1])
counter = 1
for i in props:
    print("Proportion good house quality in each target",counter,":", i)
    counter+=1

<a id='dependency'></a>

## 3 q. Dependency

The graph shows the distribution of dependency colored by the value of the Target. We can see that there isn’t a significant difference in the variable distribution depending on the household poverty level.
The rate is higher for non-vulnerable and vulnerable.


In [ ]:
plt.figure(figsize = (15, 8))
plt.style.use('fivethirtyeight')

# Color mapping
colors = OrderedDict({1: 'red', 2: 'orange', 3: 'blue', 4: 'green'})
poverty_mapping = OrderedDict({1: 'extreme', 2: 'moderate', 3: 'vulnerable', 4: 'non vulnerable'})

# For Dependency column plot against the density
for i, col in enumerate(['dependency']):
    ax = plt.subplot(4, 2, i + 1)
    # Iterate through the poverty levels
    for poverty_level, color in colors.items():
        # Plot each poverty level as a separate line
        sns.kdeplot(data.loc[data['Target'] == poverty_level, col].dropna(), 
                    ax = ax, color = color, label = poverty_mapping[poverty_level])
        
    plt.title(f'Dependency Distribution \n'); plt.xlabel(f'dependency'); plt.ylabel('Density')
plt.subplots_adjust(top = 2)

<a id='age'></a>

## 3 r. Age


Most of the population lies within the age group of 20 years to 60 years.
For every age group, non vulnerable class is exceeding in numbers
Moderate poverty came in second place


In [ ]:
plt.figure(figsize=(10,6))
plt.hist(X["age"].loc[y == 1], bins = 30, alpha = 0.5, label = "Extreme Poverty")
plt.hist(X["age"].loc[y == 2], bins = 30, alpha = 0.5, label = "Moderate Poverty")
plt.hist(X["age"].loc[y == 3], bins = 30, alpha = 0.5, label = "Vulnerable")
plt.hist(X["age"].loc[y == 4], bins = 30, alpha = 0.5, label = "Not Vulnerable")
plt.title("Distribution of Age by Poverty Level", fontsize = 22)
plt.xlabel('age', fontsize=15)
plt.legend(loc='upper right', fontsize=15)
plt.grid()
plt.show()

<a id='rent'></a>

## 3 s. Monthly Rent Payment Distribution

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(X["v2a1"].loc[y == 1], bins = 15, alpha = 0.5, label = "Extreme Poverty")
plt.hist(X["v2a1"].loc[y == 2], bins = 15, alpha = 0.5, label = "Moderate Poverty")
plt.hist(X["v2a1"].loc[y == 3], bins = 15, alpha = 0.5, label = "Vulnerable")
plt.hist(X["v2a1"].loc[y == 4], bins = 15, alpha = 0.5, label = "Not Vulnerable")
plt.title("Distribution of Monthly Rent Payments by Poverty Level", fontsize = 22)
plt.xlabel('v2a1', fontsize=15)
plt.legend(loc='upper right', fontsize=15)
plt.grid()
plt.show()

<a id='missing_rent'></a>

## 3 t. Missing Rent Value Distribution

In [ ]:
ax = sns.countplot(x=y,hue=X["rent_missing"])
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Missing Rent Distribution for Each Income Level")

In [ ]:
X = X.drop(columns=['epared1'])
X = X.drop(columns=['etecho1'])
X = X.drop(columns=['eviv1'])
X = X.drop(columns=['epared2'])
X = X.drop(columns=['etecho2'])
X = X.drop(columns=['eviv2'])
X = X.drop(columns=['epared3'])
X = X.drop(columns=['etecho3'])
X = X.drop(columns=['eviv3'])

<a id='pca'></a>
# 4. Data Reduction Techniques (PCA, KPCA & Feature Selection)
We scaled the data and performed PCA (including the number of components that captured 95% of the explainable variance).

## 4 a. PCA


In [ ]:
X = X.drop(columns=['regions'])
standard_scaler = preprocessing.StandardScaler().fit_transform(X.values)

In [ ]:
pca = PCA()
pca_components = pca.fit_transform(standard_scaler)

In [ ]:
#determine how many components to keep
counter = 0
ratio = 0
for i in range(len(pca.explained_variance_ratio_)):
    if ratio < 0.95:
        ratio += pca.explained_variance_ratio_[i]
        counter +=1
    else:
        break
print(counter)

In [ ]:
print(f'Explained Variance: {round(sum(PCA(n_components=counter).fit(pca_components).explained_variance_ratio_), 2)}%')

In [ ]:
#PCA Visualization
num_vars = len(pca.explained_variance_ratio_)
fig = plt.figure()
ax = plt.axes()
x = np.linspace(1, 121, 121)
y_pca = np.cumsum(pca.explained_variance_ratio_)
ax.plot(x, y_pca)
plt.ylabel('Percentage of Variance Explained')
plt.xlabel('Number of Principal Components')
plt.axvline(x=counter,color='k', linestyle='--')
plt.show();

In [ ]:
pca = PCA(n_components=counter)
features_norm_pca = pca.fit_transform(standard_scaler)

<a id='kpca'></a>
## 4 b. Kernel PCA

In [ ]:
kpca = KernelPCA()
kpca_components = kpca.fit_transform(standard_scaler)

In [ ]:
explained_variance = np.var(kpca_components, axis=0)
explained_variance_ratio = explained_variance / np.sum(explained_variance)

In [ ]:
#determine how many components to keep
counter = 0
ratio = 0
for i in range(len(explained_variance_ratio)):
    if ratio < 0.95:
        ratio += explained_variance_ratio[i]
        counter +=1
    else:
        break
print("95% of the explainable variance is captured in KPCA when", counter,"components are included")

In [ ]:
kpca = KernelPCA(n_components=counter, kernel='rbf', n_jobs=-1, remove_zero_eig=True)
kpca_components = kpca.fit_transform(standard_scaler)

We can test a model using KPCA components to determine the effectiveness of this data reduction strategy for this data. We will use the 

In [ ]:
X_train_kpca, X_val_kpca, y_train_kpca, y_val_kpca = train_test_split(kpca_components, y, test_size=0.30, random_state=2020)
lr_kpca = LogisticRegression(max_iter = 10000, solver='lbfgs', penalty='l2', C=0.1,multi_class="multinomial")
lr_kpca.fit(X_train_kpca,y_train_kpca)
log_reg_pred_train_kpca = lr_kpca.predict(X_train_kpca)

log_reg_pred_test_kpca = lr_kpca.predict(X_val_kpca)
print("Logistic regression with KPCA training set accuracy:",round(metrics.accuracy_score(y_true = y_train_kpca, y_pred = log_reg_pred_train_kpca),4))
print("Logistic regression with KPCA testing set accuracy:",round(metrics.accuracy_score(y_true = y_val_kpca, y_pred = log_reg_pred_test_kpca),4))

In [ ]:
print(classification_report(y_train_kpca, log_reg_pred_train_kpca, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))
print("Validation set classification report:")
print(classification_report(y_val_kpca, log_reg_pred_test_kpca, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

As we will see later, KPCA has very similar results compared to PCA. Given the similarities, rather than doing both PCA and KPCA for each model we will focus our analysis on PCA.





<a id='feature_selection'></a>
## 4 c. Feature Selection Using Feature Importance From a Logistic Regression Model

Using logistic regression, we can take *all* standardized features, split these features into train and test sets, then rank those features based on the values of the calculated coefficients. (As a note we tried multiple numbers of features to include and 75-80 seems to be ideal. This is also approximately the number of components PCA calculates to capture 95% of variance). This further indicates that for optimal performance, we need to include around 75-80 features regardless of data reduction strategy.

In [ ]:
X_train_fs, X_val_fs, y_train_fs, y_val_fs = train_test_split(standard_scaler, y, test_size=0.30, random_state=2020)

lr_fs = LogisticRegression(max_iter = 10000, solver='lbfgs', penalty='l2', C=0.1,multi_class="multinomial")
lr_fs.fit(X_train_fs,y_train_fs)
log_reg_pred_train_fs = lr_fs.predict(X_train_fs)

log_reg_pred_test_fs = lr_fs.predict(X_val_fs)
print("Logistic regression training set accuracy:",round(metrics.accuracy_score(y_true = y_train_fs, y_pred = log_reg_pred_train_fs),4))
print("Logistic regression testing set accuracy:",round(metrics.accuracy_score(y_true = y_val_fs, y_pred = log_reg_pred_test_fs),4))

In [ ]:
selector = RFE(lr_fs, 10)
selector = selector.fit(X_train_fs, y_train_fs)

In [ ]:
order = selector.ranking_
feature_ranks = []
for i in range(len(order)):
  if order[i] < 70:
    feature_ranks.append(X.columns[i])
print("Number of features included:", len(feature_ranks))

<a id='research_questions'></a>
# 5. Research Questions

<a id='research_question_1'></a>
#Research Question 1: Can we use PCA to identify the main components that determine poverty level?

As discussed earlier, we used PCA to identify the components that make up 95% of the explained variance. To determine variables that were important in the construction of the principal components, we can look at loading scores which are coefficients of the linear combination of the scaled attributes. Loading scores were moderately low, so we can say scores with absolute values of 0.15 indicate the attribute was important for that components. Below are the features with high loading scores for the first 3 components and their associated group as designated by the data dictionary built earlier

**PC1**

* **r4h3**: Features Describing Number of People Living in the Household
* **r4m2**: Features Describing Number of People Living in the Household
* **r4m3**: Features Describing Number of People Living in the Household
* **r4t1**: Features Describing Number of People Living in the Household
* **r4t2**: Features Describing Number of People Living in the Household
* **r4t3**: Features Describing Number of People Living in the Household
* **tamviv**: Features Describing Number of People Living in the Household
* **tamhog**: Variables Related to House Size
* **hhsize**: Variables Related to House Size
* **hogar_adul**: Features Describing Number of Individuals Living in Household (Within Different Age Groups)
* **hogar_total**: Features Describing Number of Individuals Living in Household (Within Different Age Groups)

**PC2**

* **v2a1**: Monthly Rent Payment
* **paredblolad**: Variables Describing Materials of Wall, Floor, or Roof
* **pisomoscer**: Variables Describing Materials of Wall, Floor, or Roof
* **pisocemento**: Variables Describing Materials of Wall, Floor, or Roof
* **cielorazo**: Variables Describing Materials of Wall, Floor, or Roof

* **escolari**: Features Describing Education Level of Head of Household & Members of the Household
* **edjefe** Features Describing Education Level of Head of Household & Members of the Household
* **edjefa** Features Describing Education Level of Head of Household & Members of the Household
* **meaneduc**: Features Describing Education Level of Head of Household & Members of the Household
* **elimbasu1**: Features Describing How Trash is Disposed
* **dependency**: Features Describing Number of Individuals Living in Household (Within Different Age Groups)

* **qmobilephone**: Features Indicating Types of Technology Owned
* **lugar1**: Features Describing Region Household is Located
* **house_good_quality**: Variables Describing Quality of House (Good,Avg,Poor)
* **bedrooms**: Features Describing Number of Rooms the Household Has
* **rooms**: Features Describing Number of Rooms the Household Has

**PC3**
* **r4h1**: Features Describing Number of People Living in the Household
* **r4h2**: Features Describing Number of People Living in the Household
* **r4m1**: Features Describing Number of People Living in the Household
* **r4t1**: Features Describing Number of People Living in the Household
* **r4t2**: Features Describing Number of People Living in the Household
* **estadocivil1**: Features Describing Status of Head of Household
* **hogar_nin**: Features Describing Number of Individuals Living in Household (Within Different Age Groups)
* **hogar_adul**: Features Describing Number of Individuals Living in Household (Within Different Age Groups)
* **hogar_mayor**: Features Describing Number of Individuals Living in Household (Within Different Age Groups)
* **instlevel1**: Features Describing Education Level of Head of Household & Members of the Household
* **bedrooms**: Features Describing Number of Rooms the Household Has
* **tipovivi1**: Features Decribing Rent/Mortgage Payment
* **tipovivi3**: Features Decribing Rent/Mortgage Payment
* **rent_missing**: Features Decribing Rent/Mortgage Payment

**Analysis of loadings**:

Based on these results, Principal Component 1 is built almost entirely from features describing the size of the household (# of people and physical size). The expectation is that these would be good variables for distinguishing poverty level: wealthier people will generally have larger homes. We may also expect poorer people to fit more people into a single home than a wealthier house would.

Principal Component 2 has mostly variables describing the house materials, education level of people in the house, as well as the region and if the overall house quality is good. Lugar1 is the region with the highest average rent payments, and the variable indicating if the house quality is good would be best at identifying more afflued households. Therefore, *the features that build PC2 are better at identifying less vulnerable households.* 

Principal Component 3 Is similar to principal component 1 but had a strong loading from the "rent_missing" attribute. The presumption is that poorer households have more difficulty in providing records regarding finances (e.g. rent), so missing values for rent may be an indicator for more impoverished household. There, *the features that build PC3 are better at identifying more vulnerable households.*



In [ ]:
pca_find_vars = PCA(n_components=3)
pca_find_vars.fit_transform(standard_scaler)
print(pd.DataFrame(pca_find_vars.components_,columns=X.columns,index = ['PC-1','PC-2', 'PC-3']))

**PCA & Feature Importance**

Additionally, we can supplement this principal component analysis with the feature importance rankings determined by the logistic regression used earlier (top 10 below). All but two were strong contributors to the first 3 principal components, indicating that these features are strong predictors of poverty

**v2a1** --> PC2

**v18q** --> Not in first 3 PCs

**r4t2** --> PC1 & PC2

**tamviv** --> PC1

**coopele** --> Not in first 3 PCs

**hogar_nin** --> PC3

**hogar_adul** --> PC3

**edjefa** --> PC2

**meaneduc** --> PC2

**qmobilephone** --> PC2

In [ ]:
for i in range(len(order)):
  if order[i] == 1:
    print(X.columns[i])

<a id='research_question_2'></a>

#Research Question 2: Can we develop a model better than our baseline model?

We decided to use a K-Nearest Neighbors algorithm to act as a baseline model. Given the imbalanced nature of the data, a KNN algorithm should fair well even without any hyperparameter tuning. The baseline model uses no data transformation (scaling, data reduction, resampling, etc).

In [ ]:
X_train_base, X_val_base, y_train_base, y_val_base = train_test_split(X, y, test_size=0.30, random_state=2020)

In [ ]:
clf_knn_base = KNeighborsClassifier()
clf_knn_base.fit(X_train_base, y_train_base)
KNN_base_fit_train = clf_knn_base.predict(X_train_base)
KNN_base_fit_val = clf_knn_base.predict(X_val_base)
print("Training set classification report:")
print(classification_report(y_train_base, KNN_base_fit_train, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))
print("Validation set classification report:")
print(classification_report(y_val_base, KNN_base_fit_val, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

There is a pretty striking difference in results between the training set and test set for the KNN model (overfitting). The baseline model indicates that the data to provide some predictive power, and we will investigate many algorithms to determine which is best at solving our problem.

<a id='research_question_3'></a>

#Research Question 3: How important are regional differences?

To understand the generalizability of the models we develop, we should understand regional differences in our data and see if predictive modeling is similar regardless of region.

First we compared identified general differences between regions based on the rent variable. We can see that Lugar1 pays the most rent while Lugar6 pays the least (on average).

In [ ]:
for i in range(7):
    for j in X.columns:
        if j == "lugar" + str(i):
            print("Average monthly rent payment for region lugar" +str(i),":",X.groupby('lugar'+str(i), as_index=False)['v2a1'].mean().iloc[1,1])

More than half of all observations come from lugar1. We will build a logistic regression model with default parameters for two subsets of the data: 1 for observations in lugar1 and another for observations not in lugar1. We will also scale the attributes.

In [ ]:
X_lugar_data = X
X_lugar_data["target"] = y

X_lugar1 = X_lugar_data[X_lugar_data["lugar1"]==1]
X_lugar_other = X_lugar_data[(X_lugar_data["lugar6"]==1) | (X_lugar_data["lugar5"]==1) | (X_lugar_data["lugar4"]==1) | (X_lugar_data["lugar3"]==1)]
print("Number of households in lugar1:",X_lugar1.shape[0])
print("Number of households not it lugar 1:", X_lugar_other.shape[0])

In [ ]:
y_lugar1 = X_lugar1["target"]
y_lugar_other = X_lugar_other["target"]
X_lugar1 = X_lugar1.drop(columns=['target'])
X_lugar_other = X_lugar_other.drop(columns=['target'])



In [ ]:
standard_scaler_lugar1 = preprocessing.StandardScaler().fit_transform(X_lugar1.values)
standard_scaler_lugar_other = preprocessing.StandardScaler().fit_transform(X_lugar_other.values)
X_train_lugar1, X_val_lugar1, y_train_lugar1, y_val_lugar1 = train_test_split(standard_scaler_lugar1, y_lugar1, test_size=0.30, random_state=2020)
X_train_lugar_other, X_val_lugar_other, y_train_lugar_other, y_val_lugar_other = train_test_split(standard_scaler_lugar_other, y_lugar_other, test_size=0.30, random_state=2020)


In [ ]:
lr_lugar1 = LogisticRegression(max_iter = 10000,multi_class="multinomial")
lr_lugar1.fit(X_train_lugar1,y_train_lugar1)
log_reg_pred_train_lugar1 = lr_lugar1.predict(X_train_lugar1)

log_reg_pred_test_lugar1 = lr_lugar1.predict(X_val_lugar1)
print("Logistic regression training set accuracy for Lugar 1:",round(metrics.accuracy_score(y_true = y_train_lugar1, y_pred = log_reg_pred_train_lugar1),4))
print("Logistic regression testing set accuracy for Lugar 1:",round(metrics.accuracy_score(y_true = y_val_lugar1, y_pred = log_reg_pred_test_lugar1),4))

In [ ]:

print(classification_report(y_train_lugar1, log_reg_pred_train_lugar1, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))
print("Validation set classification report:")
print(classification_report(y_val_lugar1, log_reg_pred_test_lugar1, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))


In [ ]:
lr_lugar_other = LogisticRegression(max_iter = 10000,multi_class="multinomial")
lr_lugar_other.fit(X_train_lugar_other,y_train_lugar_other)
log_reg_pred_train_lugar_other = lr_lugar_other.predict(X_train_lugar_other)

log_reg_pred_test_lugar_other = lr_lugar_other.predict(X_val_lugar_other)
print("Logistic regression training set accuracy for regions not in Lugar 1:",round(metrics.accuracy_score(y_true = y_train_lugar_other, y_pred = log_reg_pred_train_lugar_other),4))
print("Logistic regression testing set accuracy for regions not in Lugar 1:",round(metrics.accuracy_score(y_true = y_val_lugar_other, y_pred = log_reg_pred_test_lugar_other),4))

In [ ]:
print(classification_report(y_train_lugar_other, log_reg_pred_train_lugar_other, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))
print("Validation set classification report:")
print(classification_report(y_val_lugar_other, log_reg_pred_test_lugar_other, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

Interestingly, the logistic regression models perform better on both subsets of the data compared to when the entire data is used (as we will see later). This indicates that models may be better applied on more homogeneous groups than taking many groups and putting them together in a single dataset.

We are also interested in seeing differences in the region in a more general sense: rural vs urban. We can treat rural vs urban as a target variable and treat all other other variables as explanatory features. The plot below using the average education of members of the household shows that there will clearly be differences between rural and urban (urban areas have a much larger proportion of people with a mean education greater than 15)


In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.lmplot(x="meaneduc", y="v2a1",col = "urban", data=X)

We ran our SVC and KNN classifiers trying to predict region to see if either method could identify regional differences.

In [ ]:
y_reg = X["urban"]
X_train_reg, X_val_reg, y_train_reg, y_val_reg = train_test_split(standard_scaler, y_reg, test_size=0.30, random_state=2020)

In [ ]:
knn_urban_rural = KNeighborsClassifier()
knn_urban_rural.fit(X_train_reg, y_train_reg)
knn_urban_rural_val_pred = knn_urban_rural.predict(X_val_reg)
print(classification_report(y_val_reg, knn_urban_rural_val_pred,target_names=["Rural","Urban"]))

In [ ]:
disp = plot_confusion_matrix(knn_urban_rural, X_val_reg, y_val_reg,
                                 display_labels=["urban", "rural"],
                                cmap=plt.cm.Blues,values_format = '')
disp.ax_.set_title("Confusion matrix")
disp.im_.colorbar.remove()
print(disp)

The confusion matrix above demonstrates that urban vs rural is much easier to predict in the data than poverty (however recall is slightly weak with many false positives, possibly due to class imbalance)

## 4. Can we build a useful model using only data on the house (traditional PMT attributes)?

Given that one of the reasons this competition was created was to see how non-traditional PMT features would help in the prediction of social welfare need, we wanted to create our own traditional PMT model as a means of comparison to models that utilize traditional and non-traditional PMT features. First we need to subset the data based on these features (as determined by our data dictionary)

In [ ]:
houseFactors_subset = X.loc[:,["tamhog", "hhsize","paredblolad", "paredzocalo", "paredpreb", "pareddes" , "paredmad", "paredzinc", "paredfibras", "paredother", "pisomoscer", "pisocemento", "pisoother", "pisonatur", "pisonotiene", "pisomadera", "techozinc", "techoentrepiso", "techocane", "techootro", "cielorazo","house_poor_quality","house_medium_quality","house_good_quality","lugar1", "lugar2", "lugar3", "lugar4", "lugar5", "lugar6","urban"]]
min_max_scaler = preprocessing.MinMaxScaler().fit(houseFactors_subset.values)
features_norms = min_max_scaler.transform(houseFactors_subset.values)
X_train_PMT, X_val_PMT, y_train_PMT, y_val_PMT = train_test_split(features_norms, y, test_size=0.30, random_state=2020)

Using a train-test split of normalized PMT features, we will try two models: KNN and SVC to determine the predictive performance of these features.

**KNN with Traditional PMT Features**

In [ ]:
clf = KNeighborsClassifier()
clf.fit(X_train_PMT, y_train_PMT)

In [ ]:
KNN_Fit_PMT = clf.predict(X_val_PMT)
print('KNN PMT Classification Report: ')
print(classification_report(y_val_PMT, KNN_Fit_PMT, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

**SVC with Traditional PMT Features**

In [ ]:
svc = SVC(kernel ='rbf', C = 20, gamma = 'scale')
svc.fit(X_train_PMT, y_train_PMT)
SVC_fit_PMT = svc.predict(X_val_PMT)
print('SVC PMT Classification Report: ')
print(classification_report(y_val_PMT, SVC_fit_PMT, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

From the above Classification Reports, performance is actually quite strong (macro F-1 Average scores of 0.4 or higher are quite good for this data). Both of these models actually do fairly well at identifying classes with fewer samples (compared to other models analyzed later with recall and f-1 scores below 0.2). We will see later that the difference in KNN is minimal while SVC has a moderate drop in performance when only using traditional PMT variables. This is likely due to the fact that the parameters that we removed were critical to the boundary and thus the preformance of the classifier.


<a id='models'></a>
# 6. Models

This section includes code for running machine learning models and hyperparameter tuning. See section 6 for experiment results.

## 6 a. Train-Test Split

Models will use the PCA components calculate earlier, which are split into train and test sets. Traditional and Non-Traditional PMT features were included for this section.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(features_norm_pca, y, test_size=0.30, random_state=2020)

## 6 b. Logistic Regression
<a id='logistic_regression'></a>

We will first build a logistic regression model without gridsearch as a baseline to compare against hyperparameter tuning.

In [ ]:
lr1 = LogisticRegression(max_iter = 10000,multi_class="multinomial")
lr1.fit(X_train,y_train)
log_reg_pred_train = lr1.predict(X_train)

log_reg_pred_test = lr1.predict(X_val)
print("Logistic regression training set accuracy:",round(metrics.accuracy_score(y_true = y_train, y_pred = log_reg_pred_train),4))
print("Logistic regression testing set accuracy:",round(metrics.accuracy_score(y_true = y_val, y_pred = log_reg_pred_test),4))

In [ ]:
def get_cv_accuracies(model_fit, n_folds = 5):
    results = model_fit.cv_results_
    accuracies_lst = []
    accuracy = 0
    for i in range(len(results["mean_fit_time"])):
        for j in range(n_folds):
            accuracy += results["split"+str(j)+"_test_score"][i]
        accuracies_lst.append(accuracy/n_folds)
        accuracy = 0
    indx = accuracies_lst.index(max(accuracies_lst))
    best_param = results['params'][indx]
    
    return best_param

Logistic regression with gridsearch cv.

In [ ]:
grid={"C":np.logspace(-3,3,7)}
logreg=LogisticRegression(max_iter=10000,multi_class="multinomial",penalty= 'l2')
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train,y_train)


print("Best Parameters:",get_cv_accuracies(logreg_cv))

In [ ]:
lr1 = LogisticRegression(max_iter = 10000, solver='lbfgs', penalty='l2', C=10,multi_class="multinomial")
lr1.fit(X_train,y_train)
log_reg_pred_train = lr1.predict(X_train)
log_reg_pred_test = lr1.predict(X_val)
print("Logistic regression training set accuracy:",round(metrics.accuracy_score(y_true = y_train, y_pred = log_reg_pred_train),4))
print("Logistic regression testing set accuracy:",round(metrics.accuracy_score(y_true = y_val, y_pred = log_reg_pred_test),4))

# <a id='adaboost'></a>
## 6 c. AdaBoost Model

In [ ]:
ac = AdaBoostClassifier()
ac.fit(X_train,y_train.ravel())

pred9 = ac.predict(X_train)
pred10 = ac.predict(X_val)

In [ ]:
pred_ada_train = ac.predict(X_train) # make prediction on the training set 
pred_ada_test = ac.predict(X_val) # make prediction on the testing set 

print("The training accuacy is: ",metrics.accuracy_score(y_true = y_train, y_pred = pred_ada_train))
print("The testing accuacy is: ",metrics.accuracy_score(y_true = y_val, y_pred = pred_ada_test))

# <a id='gnb'></a>
## 6 d. Gaussian Naive Bayes Classifier 

In [ ]:
clf_nb = GaussianNB()
clf_nb.fit(X_train, y_train)

In [ ]:
pred_train = clf_nb.predict(X_train)
metrics.accuracy_score(y_true = y_train, y_pred = pred_train)
print("Gaussian Naive Bayes classifier training set accuracy:",round(metrics.accuracy_score(y_true = y_train, y_pred = pred_train),4))

In [ ]:
pred_nb = clf_nb.predict(X_val)
metrics.accuracy_score(y_true = y_val, y_pred = pred_nb)
print("Gaussian Naive Bayes classifier testing set accuracy:",round(metrics.accuracy_score(y_true = y_val, y_pred = pred_nb),4))

In [ ]:
param_grid = {
    'var_smoothing': [1e-11, 1e-10, 1e-09, 1e-08, 1e-07, 1e-5],
    }

grid_search = GridSearchCV(estimator = clf_nb, param_grid = param_grid, cv = 5)
grid_search.fit(features_norm_pca, y)

print(grid_search.best_params_)

In [ ]:
clf_nb2 = GaussianNB(var_smoothing = 1e-11)
clf_nb2.fit(X_train, np.ravel(y_train))
NB_fit_train = clf_nb2.predict(X_train)
NB_fit_CV5 = clf_nb2.predict(X_val)

<a id='decision_tree'></a>

## 6 e. Decision Tree

In [ ]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [ ]:
tree_clf = DecisionTreeClassifier()

parameters = {"min_samples_split":[50,100,200],
             "max_depth":[1,5,10,50,100]}

tree_clf.fit(X_train,y_train.ravel()) # fit the model

In [ ]:
# find the best model use scoring = balanced_accuracy
grid_cv1 = GridSearchCV(estimator=tree_clf, param_grid=parameters, cv=5,scoring="balanced_accuracy") 
grid_cv1.fit(X_train,y_train.ravel()) 

print_results(grid_cv1)

In [ ]:
# use f1_macro scoring 
grid_cv2 = GridSearchCV(estimator=tree_clf, param_grid=parameters, cv=5,scoring="f1_macro") 
grid_cv2.fit(X_train,y_train.ravel()) 

print_results(grid_cv2)

In [ ]:
dot_data = tree.export_graphviz(grid_cv2.best_estimator_, out_file=None, filled=True)
# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph

In [ ]:
# scoring = balanced_accuracy
clf1 = DecisionTreeClassifier(max_depth=50,min_samples_split=50)
clf1= clf1.fit(X_train,y_train)
y_pred1 = clf1.predict(X_val)
print("Decision Tree accuracy on testing set is:",metrics.accuracy_score(y_val, y_pred1))

In [ ]:
# scoring = f1_macro
clf2 = DecisionTreeClassifier(max_depth=100,min_samples_split=50)
clf2.fit(X_train,y_train)
y_pred_dt_train = clf2.predict(X_train)
y_pred2 = clf2.predict(X_val)
print("Decision Tree accuracy on testing set is:",metrics.accuracy_score(y_train, y_pred_dt_train))
print("Decision Tree accuracy on testing set is:",metrics.accuracy_score(y_val, y_pred2))

<a id='random_forest'></a>

## 6 f. Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train.ravel())

In [ ]:
#takes a long time to run... BEST PARAMS: {'max_depth': 50, 'min_samples_split': 50, 'n_estimators': 5}
"""
rf_parameters = {"min_samples_split":[50,100,200],
             "max_depth":[1,5,10,50,100],
             "n_estimators":[5, 50, 250, 500]}

rf_grid_cv = GridSearchCV(estimator=rf, param_grid=rf_parameters, cv=5,scoring="f1_macro") 
rf_grid_cv.fit(X_train,y_train.ravel()) # fit the model

print_results(rf_grid_cv)
"""

In [ ]:
clf_RF = RandomForestClassifier(max_depth=50,min_samples_split=50,n_estimators=5)
clf_RF = clf_RF.fit(X_train,y_train)
y_pred_rf_train = clf_RF.predict(X_train)
y_pred_rf = clf_RF.predict(X_val)
print("Random Forest accuracy on training set is:",metrics.accuracy_score(y_train, y_pred_rf_train))
print("Random Forest accuracy on testing set is:",metrics.accuracy_score(y_val, y_pred_rf))

<a id='knn'></a>

## 6 g. KNN

In [ ]:
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)

In [ ]:
KNN_fit = clf.predict(X_val)

In [ ]:
clf.get_params()

In [ ]:
#commented out because this code takes a very long time to run.
#best params: {'algorithm': 'auto', 'n_neighbors': 50, 'p': 1, 'weights': 'distance'}
#param_grid = {
#    'algorithm': ['auto', 'brute', 'ball_tree', 'kd_tree'],
#    'n_neighbors': [2, 5, 10, 20, 50],
#    'p': [ 1, 2, 3], #1 is the same as specifying euclidean distance and 2 is the same as Manhattan
#    'weights': ['uniform', 'distance']
#    }

#grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, cv = 5)
#grid_search.fit(features_norms, Y)

#print(grid_search.best_params_)

In [ ]:
clf = KNeighborsClassifier(algorithm = 'auto', n_neighbors = 50, p = 1, weights = 'distance')

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
KNN_train_pred = clf.predict(X_train)
KNN_fit = clf.predict(X_val)
print("KNN accuracy on training set is:",metrics.accuracy_score(y_train, KNN_train_pred))
print("KNN accuracy on testing set is:",metrics.accuracy_score(y_val, KNN_fit))

<a id='neural'></a>

## 6 h. Neural Network

In [ ]:
warnings.filterwarnings("ignore")

hidden_layer_range = range(10,25)
scores_mlp = {}
scores_list_mlp = []
for h in hidden_layer_range:
    mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(h,), random_state=2020,max_iter=150)
    mlp.fit(X_train, y_train) 
    y_pred=mlp.predict(X_val)
    scores_mlp[h] = accuracy_score(y_val,y_pred)
    scores_list_mlp.append(accuracy_score(y_val,y_pred))

In [ ]:
hidden_layers = scores_list_mlp.index(max(scores_list_mlp))+1
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=hidden_layers,max_iter=150, warm_start=True, random_state=1)
mlp.fit(X_train, y_train)
y_pred_nn_train=mlp.predict(X_train)
y_pred_nn=mlp.predict(X_val)
print("Training accuracy for Neural Network: ",accuracy_score(y_train,y_pred_nn_train)*100)

print("Testing accuracy for Neural Network: ",accuracy_score(y_val,y_pred_nn)*100)
warnings.resetwarnings()


<a id='sgd'></a>

## 6 i. Stochastic Gradient Descent

In [ ]:
clf_SGD = SGDClassifier()
clf_SGD.fit(X_train, y_train)

SGD_parameters = {'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], # learning rate
                #'n_iter_': [1000], # number of epochs
                'penalty': ['l2'],
                'n_jobs': [-1]}

SGD_grid_cv = GridSearchCV(estimator=clf_SGD, param_grid=SGD_parameters, cv=5,scoring="f1_macro") 
SGD_grid_cv.fit(X_train,y_train.ravel()) # fit the model

print_results(SGD_grid_cv)

In [ ]:
clf_SGD = SGDClassifier(alpha = 0.0001,n_jobs = -1, penalty = 'l2')
clf_SGD.fit(X_train, y_train)
pred_SGD_train = clf_SGD.predict(X_train)
pred_SGD = clf_SGD.predict(X_val)
print("SGD testing set accuracy:",round(metrics.accuracy_score(y_true = y_train, y_pred = pred_SGD_train),4))
print("SGD testing set accuracy:",round(metrics.accuracy_score(y_true = y_val, y_pred = pred_SGD),4))

In [ ]:
clf_SGD.fit(X_train_kpca,y_train)
sgd_kpca_pred = clf_SGD.predict(X_val_kpca)
print("SGD testing set accuracy after applying KPCA:",round(metrics.accuracy_score(y_true = y_val, y_pred = sgd_kpca_pred),4))

<a id='svc'></a>
## 6 j. Support Vector Classifier

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel ='rbf')
svc.fit(X_train, y_train)
predicted_train = svc.predict(X_train)
predicted_val = svc.predict(X_val)

In [ ]:
#param_grid = {
#    'C': [0.1, 0.5, 1, 2, 5, 10, 20],
#    'gamma': ['scale', 'auto', 1, 2, 3]
#    }

#grid_search = GridSearchCV(estimator = svc, param_grid = param_grid, scoring = 'f1_macro')
#predicted = grid_search.fit(X_train, y_train)

#print(grid_search.best_params_)

#Takes a long time to run, output: {'C': 20, 'gamma': 'scale'}


In [ ]:
svc = SVC(kernel ='rbf', C = 20, gamma = 'scale')
svc.fit(X_train, y_train)
predicted_train = svc.predict(X_train)
SVC_fit = svc.predict(X_val)

<a id='imbalanced'></a>

## 5 k. Imbalanced Class Techniques

To combat the imbalanced distribution of the target variable, we tried rebalancing techniques on a few models (logistic regression and a neural network)

## Oversampling

Below you will see that some of the accuracies are actually higher on the test set for oversampling. This has to do with the oversampling process: only the training set is oversampled to avoid data leakage. However the neural network model has high variance on the training model leading to a high discrepancy in training/testing errors.

In [ ]:
X_train_oversample = pd.DataFrame(X_train)
y_train_oversample = np.array(y_train)
X_train_oversample["y"] = y_train_oversample

count_4, count_3, count_2, count_1 = X_train_oversample["y"].value_counts()

In [ ]:
features_class_1 = X_train_oversample[y_train_oversample==1]
features_class_2 = X_train_oversample[y_train_oversample==2]
features_class_3 = X_train_oversample[y_train_oversample==3]
features_class_4 = X_train_oversample[y_train_oversample==4]

In [ ]:
features_class_1_over = features_class_1.sample(count_4,replace=True)
features_class_2_over = features_class_2.sample(count_4,replace=True)
features_class_3_over = features_class_3.sample(count_4,replace=True)
features_over_sampled = pd.concat([features_class_1_over,features_class_2_over,features_class_3_over,features_class_4],axis=0)
X_train_over = features_over_sampled.drop('y',axis=1)
y_train_over = features_over_sampled['y']

In [ ]:
clf_RF_over = RandomForestClassifier(max_depth=50,min_samples_split=50,n_estimators=5)
clf_RF_over.fit(X_train_over,y_train_over)
y_pred_rf_train_over = clf_RF.predict(X_train_over)
y_pred_rf_over = clf_RF.predict(X_val)
print("Random Forest accuracy on train set with over sampling is:",metrics.accuracy_score(y_train_over, y_pred_rf_train_over))
print("Random Forest accuracy on test set with over sampling is:",metrics.accuracy_score(y_val, y_pred_rf_over))

In [ ]:
lr1_over = LogisticRegression(max_iter = 10000, solver='lbfgs', penalty='l2', C=1,multi_class="multinomial")
lr1_over.fit(X_train_over,y_train_over)
log_reg_pred_train_over = lr1_over.predict(X_train_over)
log_reg_pred_test_over = lr1_over.predict(X_val)
print("Logistic regression with over-sampling training set accuracy:",round(metrics.accuracy_score(y_true = y_train_over, y_pred = log_reg_pred_train_over),4))
print("Logistic regression with over-sampling testing set accuracy:",round(metrics.accuracy_score(y_true = y_val, y_pred = log_reg_pred_test_over),4))


In [ ]:
warnings.filterwarnings("ignore")

hidden_layer_range = range(10,25)
scores_mlp = {}
scores_list_mlp = []
for h in hidden_layer_range:
    mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(h,), random_state=2020,max_iter=150)
    mlp.fit(X_train_over, y_train_over) 
    y_pred_over=mlp.predict(X_val)
    scores_mlp[h] = accuracy_score(y_val,y_pred_over)
    scores_list_mlp.append(accuracy_score(y_val,y_pred_over))

In [ ]:
hidden_layers = scores_list_mlp.index(max(scores_list_mlp))+1
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=hidden_layers,max_iter=150, warm_start=True, random_state=1)
mlp.fit(X_train_over, y_train_over)
y_pred_over_nn_train=mlp.predict(X_train_over)
y_pred_over_nn=mlp.predict(X_val)

print("Accuracy for Neural Network training set with over-sampling:",accuracy_score(y_train_over, y_pred_over_nn_train)*100)
print("Accuracy for Neural Network testing set with over-sampling: ",accuracy_score(y_val,y_pred_over_nn)*100)
warnings.resetwarnings()


## Undersampling

In [ ]:
features_class_2_under = features_class_2.sample(count_1,replace=True)
features_class_3_under = features_class_3.sample(count_1,replace=True)
features_class_4_under = features_class_4.sample(count_1,replace=True)

features_under_sampled = pd.concat([features_class_1,features_class_2_under,features_class_3_under,features_class_4_under],axis=0)
X_train_under = features_under_sampled.drop('y',axis=1)
y_train_under = features_under_sampled['y']


**Logistic Regression**

In [ ]:
lr1_under = LogisticRegression(max_iter = 10000, solver='lbfgs', penalty='l2', C=1,multi_class="multinomial")
lr1_under.fit(X_train_under,y_train_under)
log_reg_pred_train_under = lr1_under.predict(X_train_under)
log_reg_pred_test_under = lr1_under.predict(X_val)
print("Logistic regression training set accuracy with under-sampling:",round(metrics.accuracy_score(y_true = y_train_under, y_pred = log_reg_pred_train_under),4))
print("Logistic regression testing set accuracy with under-sampling:",round(metrics.accuracy_score(y_true = y_val, y_pred = log_reg_pred_test_under),4))


**Neural Network**

In [ ]:
warnings.filterwarnings("ignore")

hidden_layer_range = range(10,25)
scores_mlp = {}
scores_list_mlp = []
for h in hidden_layer_range:
    mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(h,), random_state=2020,max_iter=150)
    mlp.fit(X_train_under, y_train_under) 
    y_pred_under=mlp.predict(X_val)
    scores_mlp[h] = accuracy_score(y_val,y_pred_under)
    scores_list_mlp.append(accuracy_score(y_val,y_pred_under))

In [ ]:
hidden_layers = scores_list_mlp.index(max(scores_list_mlp))+1
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=hidden_layers,max_iter=150, warm_start=True, random_state=1)
mlp.fit(X_train_under, y_train_under)
y_pred_under_nn_train=mlp.predict(X_train_under)
y_pred_under_nn=mlp.predict(X_val)

print("Accuracy for Neural Network training set with under-sampling: ",accuracy_score(y_train_under,y_pred_under_nn_train)*100)
print("Accuracy for Neural Network testing set with under-sampling: ",accuracy_score(y_val,y_pred_under_nn)*100)
warnings.resetwarnings()


<a id='experiment_results'></a>

# 7 a. Experiment Results


Classification reports are included for each of the models for the test set. The main metric of interest is macro avg F-1 score (the measure used to evaluate performance in the Kaggle competition). We also include training and testing set confusion matrices.

**Logistic Regression :** 

Grid Search CV found the best parameters as max_iter = 10000, solver='lbfgs', penalty='l2', C=10,multi_class="multinomial". Macro average F-1 was 0.4 but the "Vulnerable" class had weak measures indicating there may be opportunity to improve.


In [ ]:
print(classification_report(y_val, log_reg_pred_test, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

In [ ]:
print("Logistic Regression with Over-Sampling confusion matix for training set:")
print(confusion_matrix(y_train, log_reg_pred_train))

print("Logistic Regression with Over-Sampling confusion matix for testing set:")
print(confusion_matrix(y_val, log_reg_pred_test))

**Adaboost:**

Accuracy on the testing set is 0.64. Macro average F-1 is lower than most of the other models. The vulnerable class had poor performance (similar to logistic regression).

In [ ]:
print(classification_report(y_val, pred10, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

In [ ]:
print("Adaboost confusion matix for training set:")
print(confusion_matrix(y_train, pred9))

print("Adaboost confusion matix for testing set:")
print(confusion_matrix(y_val, pred10))

**Gaussian Naive Bayes:**

The accuracy we got on the testing set is 0.57 after running GridSearch trying to find the best parameter for var_smoothing. The macro average is 0.34. While GNB runs rather quickly, model performance is not as good as other models we used.


In [ ]:
print(classification_report(y_val, NB_fit_CV5, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

In [ ]:
print("GNB confusion matix for training set:")
print(confusion_matrix(y_train, NB_fit_train))

print("GNB confusion matix for testing set:")
print(confusion_matrix(y_val, NB_fit_CV5))

**Decision Tree:**

Using GridSearch for hyperparameter tuning, the best model with the current data used max_depth = 100 and min_samples_split = 50. While accuracy was slightly lower than logistic regression, decision tree was actually more sensitive to classes with lower samples. F-1 macro average was 0.43, an improvement from logistic regression. The "Vulnerable" class was much better than logistic regression.

In [ ]:
print(classification_report(y_val, y_pred2,target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

In [ ]:
print("Decision Tree confusion matix for training set:")
print(confusion_matrix(y_train, y_pred_dt_train))

print("Decision Tree confusion matix for testing set:")
print(confusion_matrix(y_val, y_pred2))

**Random Forest:** 

The best model used a max_depth = 50, min_samples_split = 50, and n-estimators = 5. The accuracy on the testing set was higher but other metrics were worse than the decision tree. Given the extremely slow process of hyperparameter tuning and worse performance on random forest (possibly due to overfitting), random forests may not be a useful model going forward.

In [ ]:
print(classification_report(y_val, y_pred_rf,target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

In [ ]:
print("Random Forest confusion matix for training set:")
print(confusion_matrix(y_train, y_pred_rf_train))

print("Random Forest confusion matix for testing set:")
print(confusion_matrix(y_val, y_pred_rf))

**K-NN:**

KNN seems to have the best performance with this data. Without Gridsearch CV, all classes have very good performance (relative to other models) with macro average F-1 of 0.6 and accuracy of 0.74

In [ ]:
print(classification_report(y_val, KNN_fit, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

In [ ]:
print("KNN confusion matix for training set:")
print(confusion_matrix(y_train, KNN_train_pred))

print("KNN confusion matix for testing set:")
print(confusion_matrix(y_val, KNN_fit))

**Neural Network**


In [ ]:
print(classification_report(y_val, y_pred_nn, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

In [ ]:
print("Neural Network confusion matix for training set:")
print(confusion_matrix(y_train, y_pred_nn_train))

print("Neural Network confusion matix for testing set:")
print(confusion_matrix(y_val, y_pred_nn))

**Stochastic Gradient Descent**

In [ ]:
print(classification_report(y_val, pred_SGD.round(), target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

In [ ]:
print("SGD confusion matix for training set:")
print(confusion_matrix(y_train, pred_SGD_train))

print("SGD confusion matix for testing set:")
print(confusion_matrix(y_val, pred_SGD))

**Support Vector Classifier**

In [ ]:
print(classification_report(y_val, SVC_fit.round(), target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

In [ ]:
confusion_matrix_svc =  pd.crosstab(index=np.ravel(y_train), columns=predicted_train.ravel(), rownames=['Expected'], colnames=['Predicted'])
sns.heatmap(confusion_matrix_svc, annot=True, square=False, fmt='', cbar=False)
plt.title("Confusion Matrix (Training Set)", fontsize = 15)
plt.show()

In [ ]:
confusion_matrix_svc =  pd.crosstab(index=np.ravel(y_val), columns=SVC_fit.ravel(), rownames=['Expected'], colnames=['Predicted'])
sns.heatmap(confusion_matrix_svc, annot=True, square=False, fmt='', cbar=False)
plt.title("Confusion Matrix (Validation Set)", fontsize = 15)
plt.show()

<a id='experiment_results_b'></a>

# 7 b. Experiment Results for Models Using Class Rebalancing Techniques (select models)

Oversampling generally performed better than oversampling. Oversampling helped logistic regression slightly while oversampling hindered neural network performance.

**Logistic Regression (oversampling)**

In [ ]:
print(classification_report(y_val, log_reg_pred_test_over, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

**Neural Network (oversampling)**

In [ ]:
print(classification_report(y_val, y_pred_over_nn, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

**Logistic Regression (undersampling)**

In [ ]:
print(classification_report(y_val, log_reg_pred_test_under, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

**Neural Network (undersampling)**

In [ ]:
print(classification_report(y_val, y_pred_under_nn, target_names=["Extreme Poverty","Moderate Poverty","Vulnerable","Not Vulnerable"]))

<a id='conclusion'></a>

# 8. Conclusion

To summarize this analysis and refer back to our original research questions, we were able to identify features that made up the first few principal components in our PCA. Components tended to have high loading values from features that were in similar groups as specified by our data dictionary. We also determined that certain principal components are likely more useful in identifying affluent households while other principal components are more useful in identifying impoverished households.

We developed multiple models that perform better than a baseline model without hyperparameter tuning. Data reduction, rebalancing techniques, and hyperparameter tuning generally help model performance. Certain algorithms also perform better than others for a multitude of reasons, which we examined extensively in our experiment results section.

For regional differences (our third research question) we identified that models build on subsets of the data split on the region helped with model performance compared to using all of the data together. This is an important discovery: this indicates that to maximize generalizability of this model, the model should be applied to homogeneous groups when possible rather than treating all observations as equal. We also demonstrated that the data can predict if a household is in a rural or urban environment, indicating stark differences in households between urban and rural areas.

For our fourth research question (model differences between traditional PMT features vs all features), KNN actually had better performance when only using traditional PMT features while SVC had much better performance when including all features. Based on our results from regional differences, this seems to be a result of differences in similarity between different groups of observations (i.e. observations from different regions). For example, perhaps in a more affluent region, people who are well off tend to have more kids and therefore more people living in the house since finances are not a worry for them. In poorer regions where access to birth control may not be available, more children would be indicative of poverty. It seems the conflation of non-homogenous groups hurts the predictive power of most models, but SVC manages to do well even when including all features. This might be because the conflation of non-homogeneuous groups does not significantly impact the data points where the decision boundary is located. Therefore, **SVC would be considered our best model**: it has the highest overall metrics (F-1 Macro average, recall and precision), does much better with additional features, and appears to be the most generalizable if social welfare needs are to be predicted in other countries.

**What more could you do on this data if you had more time?**

With more time, we could have tried more models on the Kaggle holdout set. We tried some with moderate success, but as we gained more insights based on findings from our research questions, it became more clear on approaches we could have taken to have a successful model on the Kaggle holdout test set. Given the Kaggle competition is a Kernel only competition, making drastic changes required essentially making an entire other notebook which takes a lot of time.

It would also be interesting to apply the knowledge we gained to other PMT related datasets to see if findings are similar.










    


<a id='contributions'></a>

# 9. Contributions

<a id='max_rodrigues'></a>

## 9 a. Max Rodrigues
* Wrote code to handle redundant variables and missing values (handled missing values by both KNN imputation and by taking averages of aggregations of data). 
* Did exploratory analysis (correlation plot, scatterplots, countplots and lmplots)
* Wrote code for PCA/Feature Selection
* logistic regression with & without gridsearch CV neural network. 
* Applied rebalancing techniques.
* Made updates to data dictionary to be more concise.
* Wrote/helped with abstract, introduction, conclusion.
* Analyzed PCA loadings and how they related to feature importance (from logistic regression)
* Helped with research questions (built models that demonstrate some models perform worse with the conflation of dissimilar regions)
* Cleaned and organized code in the notebook
* found reference that described PMT in detail, and reference describing why KNN handles imbalanced data well



<a id='andrew_schiek'></a>

## 9 b. Andrew Schiek
Andrew Schiek: I have contributed to the exploratory analysis, machine learning applications, and the research questions. For the exploratory analysis I examined histograms of the float variables. For machine learning applications I developed the models for the Guassian Naive Bayes, K-Nearest Neighbors, and SVC algorithms. I created the baseline and grid search for each as well as doing the analysis. I answered each of the research questions and developed the models associated, which included but was not limited to the non-observable parameter models. Additionally I contributed to the conclusion for the write-up


<a id='Xintong_Li'></a>

## 9 c. Xintong Li
* Preparing Data: Data Cleaning
* Exploratory Analysis: PCA (not included in the report) and KPCA and Histograms
* Machine Learning Models: Decison Tree, Random Forest, SGD
* Feature Importance
* Expriment Results: Summarizing ML models and results

<a id='Mashall_Jahangir'></a>

## 9 d. Mashall Jahangir
- Exploratory Analysis :
    Pie Chart, Boxplots and Variable Distribution
- Machine Learning Model :
    Gaussian Naive Bayes Classifier, Random Forest and Adaboost Model
- Overall : Helped writing the notebook sections data dictionary, eda, references, abstract.

<a id='references'></a>
## 10. References


1.   Zhang, J.P. and Mani, I. (2003) KNN Approach to Unbalanced Data Distributions: A Case Study Involving Information Extraction. Proceeding of International Conference on Machine Learning (ICML 2003), Workshop on Learning from Imbalanced Data Sets, Washington DC, 21 August 2003.

2.   Social Protection & Labor team (2010). Measuring income and poverty using Proxy Means Tests. Retrieved from https://olc.worldbank.org/sites/default/files/1.pdf

3. Adrian G. Rodriguez, Stephen M. Smith, A comparison of determinants of urban, rural and farm poverty in Costa Rica, World Development, Volume 22, Issue 3,1994,Pages 381-397,ISSN 0305-750X,https://doi.org/10.1016/0305-750X(94)90129-5.

4. J. H. Mohamud and O. N. Gerek, "Poverty Level Characterization via Feature Selection and Machine Learning," 2019 27th Signal Processing and Communications Applications Conference (SIU), Sivas, Turkey, 2019, pp. 1-4, doi: 10.1109/SIU.2019.8806548.